In [10]:
# This script requires the following libraries:
# pip install gradio transformers torch pytesseract
# You may also need to install Tesseract OCR on your system.
# For Debian/Ubuntu: sudo apt update && sudo apt install tesseract-ocr
# For macOS: brew install tesseract
# For Windows: Download from https://github.com/UB-Mannheim/tesseract/wiki

import gradio as gr
import re
from transformers import pipeline
import pytesseract
from PIL import Image

def get_ner_pipeline():
    """
    Initializes a Named Entity Recognition (NER) pipeline from Hugging Face.
    This function loads the model only once when the application starts.
    """
    try:
        # We use a general-purpose NER model. In a real-world scenario, a model
        # fine-tuned on medical text would be used.
        return pipeline("ner", model="dslim/bert-base-NER")
    except Exception as e:
        print(f"Error loading the Hugging Face model: {e}")
        return None

# Load the model once at the beginning of the script
ner_pipeline = get_ner_pipeline()
if not ner_pipeline:
    raise RuntimeError("Could not load the Hugging Face model. Exiting.")

def extract_drug_info(prescription_text):
    """
    Extracts drug names from the prescription text using a Hugging Face NER model.

    Args:
        prescription_text (str): The raw text of the patient's prescription.

    Returns:
        list: A list of unique drug names found in the text.
    """
    if not ner_pipeline:
        return []

    entities = ner_pipeline(prescription_text)
    found_drugs = set()

    known_drugs = {"aspirin", "warfarin", "paracetamol", "ibuprofen", "cetirizine", "loratadine", "amoxicillin"}

    for entity in entities:
        word = entity['word'].replace('##', '')
        if entity['entity_group'] not in ['PER', 'LOC', 'ORG'] and word.lower() in known_drugs:
            found_drugs.add(word)

    for drug in known_drugs:
        if re.search(r'\b' + re.escape(drug) + r'\b', prescription_text, re.IGNORECASE):
            found_drugs.add(drug)


    return list(found_drugs)

def check_drug_interactions(drugs):
    """
    Detects and flags harmful drug interactions.

    In a production system, this function would make an API call to a
    clinical decision support system (like one offered by IBM Watson)
    to get real-time, comprehensive interaction data. This is a
    hardcoded example for demonstration.
    Args:
        drugs (list): A list of drug names to check.

    Returns:
        dict: A map of harmful interaction pairs to their descriptions.
    """
    interactions = {}
    drugs_lower = [drug.lower() for drug in drugs]

    if "aspirin" in drugs_lower and "warfarin" in drugs_lower:
        interactions["Aspirin + Warfarin"] = (
            "Major interaction: High risk of serious bleeding. Both are anticoagulants, "
            "and their combined effect can lead to severe hemorrhage."
        )
    if "ibuprofen" in drugs_lower and "warfarin" in drugs_lower:
        interactions["Ibuprofen + Warfarin"] = (
            "Major interaction: Increased risk of gastrointestinal bleeding. Both drugs can "
            "irritate the stomach lining and thin the blood, leading to a higher risk of hemorrhage."
        )
    return interactions

def suggest_dosage(drug_name, patient_age):
    """
    Provides age-specific dosage recommendations.

    In a production system, this would query a database of drug safety profiles
    and clinical guidelines. This is a simplified example to show the concept.

    Args:
        drug_name (str): The name of the drug.
        patient_age (int): The age of the patient.

    Returns:
        str: A string containing the dosage suggestion.
    """
    drug_name_lower = drug_name.lower()
    if drug_name_lower == "aspirin":
        if patient_age < 16:
            return f"Dosage for {drug_name}: Avoid in patients under 16 due to risk of Reye's syndrome. Consult a physician."
        else:
            return f"Dosage for {drug_name}: Typically 75-100mg daily for antiplatelet effect, or 300-900mg every 4-6 hours for pain/fever. Consult a physician for specific dosage based on condition."
    elif drug_name_lower == "warfarin":
        return f"Dosage for {drug_name}: Dosage is highly individualized based on INR levels. Consult a physician for precise dosage and monitoring."
    elif drug_name_lower == "paracetamol":
        if patient_age < 2:
            return f"Dosage for {drug_name}: Consult a physician. Dosage is based on weight."
        elif patient_age < 12:
            return f"Dosage for {drug_name}: Dosage based on weight, typically 10-15 mg/kg every 4-6 hours. Consult a physician."
        else:
            return f"Dosage for {drug_name}: 500-1000mg every 4-6 hours, maximum 4000mg in 24 hours. Consult a physician."
    elif drug_name_lower == "ibuprofen":
         if patient_age < 6:
            return f"Dosage for {drug_name}: Consult a physician. Dosage is based on weight."
         elif patient_age < 12:
            return f"Dosage for {drug_name}: Dosage based on weight, typically 5-10 mg/kg every 6-8 hours. Consult a physician."
         else:
            return f"Dosage for {drug_name}: 200-400mg every 4-6 hours, maximum 1200mg in 24 hours. Consult a physician."
    elif drug_name_lower == "cetirizine":
        if patient_age < 6:
            return f"Dosage for {drug_name}: Consult a physician. Dosage is based on age and weight."
        elif patient_age < 12:
            return f"Dosage for {drug_name}: 5-10mg daily. Consult a physician."
        else:
            return f"Dosage for {drug_name}: 10mg daily. Consult a physician."
    elif drug_name_lower == "loratadine":
        if patient_age < 2:
            return f"Dosage for {drug_name}: Consult a physician. Dosage is based on weight."
        elif patient_age < 6:
            return f"Dosage for {drug_name}: 5mg daily. Consult a physician."
        else:
            return f"Dosage for {drug_name}: 10mg daily. Consult a physician."
    elif drug_name_lower == "amoxicillin":
        return f"Dosage for {drug_name}: Dosage varies greatly based on infection type, severity, and patient age/weight. Consult a physician for specific dosage."
    else:
        return (
            f"Dosage for {drug_name}: Consult a physician for a patient of age {patient_age}. "
            "Dosage varies based on health conditions."
        )

def get_alternative(interaction_pair):
    """
    Suggests safer alternative medications when an interaction is identified.

    In a real system, a recommendation engine or a large language model would
    provide evidence-based alternatives.

    Args:
        interaction_pair (str): The names of the interacting drugs.

    Returns:
        str: A string with a suggested alternative.
    """
    if "Aspirin + Warfarin" in interaction_pair:
        return "Consider a different anti-coagulant or anti-platelet medication after clinical review, such as Clopidogrel."
    if "Ibuprofen + Warfarin" in interaction_pair:
        return "Consider using Paracetamol for pain relief, as it does not interact with Warfarin in the same way."

    return "No specific alternative suggestion for this interaction."

def analyze_prescription(prescription_input, patient_age):
    """
    Main analysis function for Gradio. This orchestrates the entire process.
    Handles both text and image input.
    """
    prescription_text = ""
    if isinstance(prescription_input, str):
        prescription_text = prescription_input
    elif prescription_input is not None:
        # In a real application, you would add code here to perform OCR on the image
        # to convert it to text. For this example, we'll just indicate that an image was received.
        return "Image received. OCR functionality is not yet implemented in this example."

    if not prescription_text:
        return "Please enter some prescription text or upload an image to analyze."

    drugs = extract_drug_info(prescription_text)

    result_markdown = "## Analysis Results\n\n"

    # 1. Drug Extraction
    result_markdown += "### Drugs Extracted\n"
    if not drugs:
        result_markdown += "No known drugs detected.\n\n"
    else:
        result_markdown += "Successfully extracted drugs: " + ", ".join(f"**`{drug}`**" for drug in drugs) + "\n\n"

    # 2. Drug Interactions
    result_markdown += "### Detected Drug Interactions\n"
    interactions = check_drug_interactions(drugs)
    if not interactions:
        result_markdown += "No significant drug-drug interactions detected.\n\n"
    else:
        for pair, description in interactions.items():
            result_markdown += f"**<font color='red'>Warning: {pair} Interaction</font>**\n"
            result_markdown += f"**Description:** {description}\n\n"
            result_markdown += f"**Suggested Alternative:** {get_alternative(pair)}\n\n"

    # 3. Dosage Recommendations
    result_markdown += "### Dosage Recommendations\n"
    if not drugs:
        result_markdown += "No dosage suggestions available."
    else:
        for drug in drugs:
            result_markdown += f" - {suggest_dosage(drug, patient_age)}\n"

    return result_markdown

def analyze_image(image_input, patient_age):
    """
    Handles image input for analysis.
    """
    if image_input is not None:
        try:
            # Perform OCR on the image
            prescription_text = pytesseract.image_to_string(image_input)
            # Pass the extracted text to the analyze_prescription function
            return analyze_prescription(prescription_text, patient_age)
        except Exception as e:
            return f"Error performing OCR: {e}"
    else:
        return "No image uploaded."


# --- Gradio Frontend ---
with gr.Blocks(title="AI Medical Prescription Verifier") as demo:
    gr.Markdown("# 💊 AI Medical Prescription Verifier")
    gr.Markdown("This tool analyzes medical prescriptions to detect harmful drug interactions and suggest age-appropriate dosages and alternatives.")

    with gr.Row():
        prescription_input = gr.Textbox(
            label="Enter Prescription Text",
            placeholder="e.g., Take aspirin 100mg and warfarin 5mg daily for heart health. Also, take paracetamol as needed.",
            lines=5,
            elem_id="prescription-text"
        )
        image_input = gr.Image(
            label="Upload Prescription Image (Optional)",
            type="pil",
            elem_id="prescription-image"
        )
        age_input = gr.Number(
            label="Patient Age",
            value=30,
            elem_id="patient-age"
        )

    analyze_button = gr.Button("Analyze Prescription", variant="primary")

    output_markdown = gr.Markdown(elem_id="output-markdown")

    analyze_button.click(
        fn=analyze_prescription,
        inputs=[prescription_input, age_input],
        outputs=output_markdown
    )
    # Add click event for image input
    image_input.upload(
        fn=analyze_image, # Call the new analyze_image function for image uploads
        inputs=[image_input, age_input],
        outputs=output_markdown
    )


if __name__ == "__main__":
    demo.launch()

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bc7782c2d654843afc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
